## Libraries

In [9]:
import pandas as pd
import numpy as np
import math
import requests
import os
import zipfile

In [10]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov
from pypfopt import objective_functions

## Download CVM files

In [11]:
# set current directory and change to folder "dados_cvm"
current_dir_orig = os.getcwd()
os.chdir(f"{current_dir_orig}/dados_cvm")

In [12]:
def download_files(years, months_2021, months_2022, url_root, url_root2):
    for year in years:
        download = requests.get(url_root + f"inf_diario_fi_{year}.zip")
        open(f"inf_diario_fi_{year}.zip", "wb").write(download.content)

    for month in months_2021:
        download = requests.get(url_root2 + f"inf_diario_fi_{month}.zip")
        open(f"inf_diario_fi_{month}.zip", "wb").write(download.content)

    for month in months_2022:
        download = requests.get(url_root2 + f"inf_diario_fi_{month}.zip")
        open(f"inf_diario_fi_{month}.zip", "wb").write(download.content)        


# years in which the files will be downloaded
years = range(2014,2021)
months_2021 = range(202101,202113)
months_2022 = range(202201,202211)
years_hist = list(range(2014,2023))

url_root = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/"
url_root2 = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/"

current_dir = os.getcwd()

# criteria to download only once
if len(os.listdir(current_dir)) == 0:
    download_files(years, months_2021, months_2022, url_root, url_root2)
else:
    pass


## Extract data from downloaded files

In [13]:
file_list = []

for file in os.listdir(current_dir):
    
        file_zip = zipfile.ZipFile(file)
        
        for sheet in file_zip.namelist():
            cotation = pd.read_csv(file_zip.open(sheet), sep=";", usecols=["CNPJ_FUNDO", "DT_COMPTC","VL_QUOTA"])
            file_list.append(cotation)

# change to original directory
os.chdir(current_dir_orig)

## Check data types

In [14]:
df_cons = pd.concat(file_list, ignore_index=True)

In [15]:
df_cons

,CNPJ_FUNDO,DT_COMPTC,VL_QUOTA
0,00.000.746/0001-03,2014-01-02,206.743553
1,00.000.746/0001-03,2014-01-03,206.564145
2,00.000.746/0001-03,2014-01-06,206.272095
3,00.000.746/0001-03,2014-01-07,204.610329
4,00.000.746/0001-03,2014-01-08,205.138251
...,...,...,...
35055802,97.929.213/0001-34,2022-10-04,10.103679
35055803,97.929.213/0001-34,2022-10-05,10.088738
35055804,97.929.213/0001-34,2022-10-06,10.055427
35055805,97.929.213/0001-34,2022-10-07,10.074393


In [16]:
df_cons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35055807 entries, 0 to 35055806
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   CNPJ_FUNDO  object 
 1   DT_COMPTC   object 
 2   VL_QUOTA    float64
dtypes: float64(1), object(2)
memory usage: 802.4+ MB


## Select relevant data

In [17]:
def data_funds(cnpj, years_hist, file_list):
        
    # consolidates dataframes by year
    count = list(range(len(years_hist)))
    dic_years = dict(zip(years_hist[:-1], count[:-1]))
    dic_df = {year:pd.concat(file_list[val*12:(val+1)*12], ignore_index=True) for year,val in dic_years.items()}
    dic_df[years_hist[-1]] = pd.concat(file_list[count[-1]*12:], ignore_index=True)   
    
    
    # filters dataframes by cnpj
    for year,df in dic_df.items():
        df_filt = df.loc[df["CNPJ_FUNDO"] == cnpj, ["CNPJ_FUNDO","DT_COMPTC","VL_QUOTA"]]
        dic_df[year] = [list(df_filt['DT_COMPTC']), list(df_filt['VL_QUOTA'])]

    # create two lists, one with date and one with price
    date = []
    price = []
    for year,value in dic_df.items():
        date = date + value[0]
        price = price + value[1]

    return date, price
        

In [18]:
# import spreadsheet with investment funds and cnpj's
funds_cnpj_df = pd.read_excel(r'./planilha_fundos/FIA1.xlsx')

# dictionary with cnpj's as keys and fund names as values
dic = dict(zip(funds_cnpj_df['CNPJ'], funds_cnpj_df['FUNDS']))

cnpj = funds_cnpj_df.loc[0,"CNPJ"]
date_comp_hist_2014_2022, vl_quote_hist_2014_2022 = data_funds(cnpj, years_hist, file_list)

# create dataframe with the column "Date"
funds_date_df = pd.DataFrame(date_comp_hist_2014_2022, columns=["Date"])

# fill dataframe with the prices per date
for cnpj,fund in dic.items():
    
    _, funds_date_df[fund] = data_funds(cnpj, years_hist, file_list)


In [19]:
# delete dataframe and list to free memory
del df_cons, file_list

In [20]:
funds_date_df

,Date,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,Atmos Ações FIC FIA,AZ Quest Small Mid Caps FIC FIA,Bahia AM Smid Caps Valor FIC FIA,BNP Paribas Small Caps FIA,Bogari Value FIC FIA,...,Opportunity Selection FIC FIA,Pacifico Ações FIC FIA,Real Investor FIC FIA BDR Nível I,Sharp Equity Value Feeder FIC FIA,SPX Apache FIC FIA,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA
0,2014-01-02,0.972803,1.052521,26.242520,1.404228,222.223048,2.402721,3.343159,147.097705,1330.855445,...,0.948812,1.288173,4.386377,1.406832,1.076706,288.387522,138.398698,2.486721,74.647206,3.561075
1,2014-01-03,0.970358,1.049983,26.244456,1.397437,221.133029,2.417377,3.341569,147.353836,1333.380199,...,0.942860,1.283497,4.417231,1.394366,1.077231,286.317087,138.871692,2.479189,77.028881,3.538951
2,2014-01-06,0.955436,1.044785,26.152192,1.388537,219.822038,2.400223,3.309387,146.636981,1328.905181,...,0.943577,1.276330,4.402191,1.392069,1.074992,284.524832,138.658790,2.471023,77.406432,3.528675
3,2014-01-07,0.951345,1.036476,25.980846,1.392356,219.808265,2.389970,3.285334,145.717513,1322.296918,...,0.935928,1.272822,4.417190,1.389634,1.065871,283.515761,138.187014,2.463331,75.519390,3.506183
4,2014-01-08,0.960561,1.037600,26.083057,1.401129,220.613639,2.385749,3.284673,146.134072,1320.079915,...,0.941475,1.275014,4.417246,1.393827,1.069235,284.908449,138.398698,2.466854,75.645788,3.528830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,2022-10-04,2.660155,2.447933,70.790486,2.777882,819.098087,6.200375,7.054847,335.539413,3290.487456,...,2.268286,3.237858,16.495505,4.012233,2.876146,790.322840,411.489072,5.562801,186.492465,12.352106
2199,2022-10-05,2.683836,2.466490,71.171714,2.779496,818.924679,6.225530,7.079094,336.112202,3293.131242,...,2.302405,3.266213,16.544227,4.038069,2.876289,793.694232,414.839965,5.551658,187.227322,12.417251
2200,2022-10-06,2.718091,2.473287,71.446229,2.816421,824.459743,6.262937,7.156142,339.164151,3328.216469,...,2.331408,3.283270,16.648048,4.049215,2.897057,797.879904,417.394793,5.573521,188.102574,12.538203
2201,2022-10-07,2.679508,2.446277,70.792442,2.754369,803.126789,6.206316,7.081880,335.067268,3262.061120,...,2.290367,3.252631,16.509900,4.009748,2.865642,786.004449,415.760540,5.461500,186.704075,12.464343


In [21]:
funds_date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2203 entries, 0 to 2202
Data columns (total 33 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Date                                       2203 non-null   object 
 1   Alaska Black FIC FIA BDR Nível I           2203 non-null   float64
 2   Apex Ações 30 FIC FIA                      2203 non-null   float64
 3   ARX Income FIC FIA                         2203 non-null   float64
 4   Atlas One FIC FIA                          2203 non-null   float64
 5   Atmos Ações FIC FIA                        2203 non-null   float64
 6   AZ Quest Small Mid Caps FIC FIA            2203 non-null   float64
 7   Bahia AM Smid Caps Valor FIC FIA           2203 non-null   float64
 8   BNP Paribas Small Caps FIA                 2203 non-null   float64
 9   Bogari Value FIC FIA                       2203 non-null   float64
 10  Brasil Capital FIC FIA  

## Risk, return and correlations

In [22]:
# drop dataframe column "Date" and set prices as float
funds_date_df2 = funds_date_df.drop('Date', axis=1).astype(float)

# splits the dataframe into periods
yrs_div = [252, 503, 754, 1003, 1252, 1505, 1756, 2008, len(funds_date_df2)]
funds_date_2016 = funds_date_df2.iloc[yrs_div[1]:yrs_div[2], :]
funds_date_2017 = funds_date_df2.iloc[yrs_div[2]:yrs_div[3], :]
funds_date_2018 = funds_date_df2.iloc[yrs_div[3]:yrs_div[4], :]
funds_date_2019 = funds_date_df2.iloc[yrs_div[4]:yrs_div[5], :]
funds_date_2020 = funds_date_df2.iloc[yrs_div[5]:yrs_div[6], :]
funds_date_2021 = funds_date_df2.iloc[yrs_div[6]:yrs_div[7], :]
funds_date_2022 = funds_date_df2.iloc[yrs_div[7]:, :]

funds_date_2014_2015 = funds_date_df2.iloc[:yrs_div[1], :]
funds_date_2015_2016 = funds_date_df2.iloc[yrs_div[0]:yrs_div[2], :]
funds_date_2016_2017 = funds_date_df2.iloc[yrs_div[1]:yrs_div[3], :]
funds_date_2017_2018 = funds_date_df2.iloc[yrs_div[2]:yrs_div[4], :]
funds_date_2018_2019 = funds_date_df2.iloc[yrs_div[3]:yrs_div[5], :]
funds_date_2019_2020 = funds_date_df2.iloc[yrs_div[4]:yrs_div[6], :]
funds_date_2020_2021 = funds_date_df2.iloc[yrs_div[5]:yrs_div[7], :]

dataframes_list = [[funds_date_2014_2015, funds_date_2016],
                   [funds_date_2015_2016, funds_date_2017],
                   [funds_date_2016_2017, funds_date_2018],
                   [funds_date_2017_2018, funds_date_2019],
                   [funds_date_2018_2019, funds_date_2020],
                   [funds_date_2019_2020, funds_date_2021],
                   [funds_date_2020_2021, funds_date_2022]]

periods = [20142015, 20152016, 20162017, 20172018, 20182019, 20192020, 20202021]

# creates a dictionary with windows and its periods
windows_periods = [periods[i:(i+4)] for i in range(len(periods)-3)]
windows = list(range(len(windows_periods)))
windows_dict = dict(zip(windows, windows_periods))

# creates a dictionary with periods
funds_date_dict = dict(zip(periods, dataframes_list))

# creates a dictionary with the daily variation in each period
daily_var_dict = {key:[value[0].pct_change(), value[1].pct_change()] for key,value in funds_date_dict.items()}
var_corr = [[corr[0], corr[1]] for corr in daily_var_dict.values()]

# creates a dictionary with the correlations in each period
corr_dict = {key:[value[0].corr(),value[1].corr()] for key,value in zip(periods,var_corr)}

# creates a dictionary with the anualized returns in each period
funds_return_dict = {key:[dict(zip(funds_date_df2.columns, mean_historical_return(value[0]).values)),
                          dict(zip(funds_date_df2.columns, mean_historical_return(value[1]).values))] 
                     for key,value in funds_date_dict.items()}

# creates a dictionary with the anualized risks in each period
funds_risk_dict = {key:[dict(zip(funds_date_df2.columns, np.sqrt(np.diagonal(sample_cov(value[0]).values)))),
                        dict(zip(funds_date_df2.columns, np.sqrt(np.diagonal(sample_cov(value[1]).values))))] 
                   for key,value in funds_date_dict.items()}

# creates a dictionary with the risk free rate (CDI) in each period
rf_rate_dict = dict(zip(periods,[[.12018, .14000],
                                 [.13618, .09930],
                                 [.11940, .06420],
                                 [.08157, .05960],
                                 [.06188, .02760],
                                 [.04345, .04420],
                                 [.03585, .09300]]))


In [23]:
funds_date_2016_2017

,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,Atmos Ações FIC FIA,AZ Quest Small Mid Caps FIC FIA,Bahia AM Smid Caps Valor FIC FIA,BNP Paribas Small Caps FIA,Bogari Value FIC FIA,Brasil Capital FIC FIA,...,Opportunity Selection FIC FIA,Pacifico Ações FIC FIA,Real Investor FIC FIA BDR Nível I,Sharp Equity Value Feeder FIC FIA,SPX Apache FIC FIA,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA
503,0.633840,0.957380,23.666944,1.521140,290.223598,2.197201,3.012552,126.376229,1347.086626,5.187793,...,0.933847,1.169256,2.973753,1.501072,0.997470,267.816480,141.984764,2.672262,33.916083,3.471809
504,0.642992,0.976034,23.721192,1.538444,294.227034,2.211713,3.056002,126.611779,1357.986295,5.222415,...,0.950539,1.191823,3.019828,1.524265,1.004303,273.082959,142.327048,2.708249,33.885720,3.531636
505,0.645767,0.971273,23.562520,1.538734,293.799957,2.219011,3.031527,125.368146,1355.376410,5.218857,...,0.952838,1.191529,2.991641,1.521953,0.997241,272.383578,141.232658,2.704085,33.331979,3.503913
506,0.617704,0.952163,22.853915,1.510971,292.082211,2.196554,2.955317,122.548307,1338.296083,5.129427,...,0.934801,1.171536,2.989703,1.509669,0.979796,267.570557,137.890306,2.679062,32.620988,3.425982
507,0.630119,0.953699,22.717478,1.509857,292.916815,2.199332,2.950439,121.990296,1337.771099,5.098315,...,0.935341,1.179494,2.996857,1.507634,0.979212,268.431820,137.627702,2.681617,32.501513,3.409287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,2.615286,1.583662,39.213075,2.022646,439.314316,3.699051,5.136197,251.755239,2241.963419,9.599722,...,1.474783,1.987637,7.530629,2.333384,1.859285,431.886802,237.635253,4.033329,85.412045,6.600845
999,2.655976,1.591504,39.642000,2.038078,441.237423,3.714695,5.178349,252.661642,2252.951086,9.664097,...,1.484328,1.995180,7.593443,2.344353,1.871607,433.443294,240.124473,4.057694,85.599497,6.665735
1000,2.680861,1.608936,39.759439,2.051211,444.623257,3.748525,5.212815,254.687115,2270.652186,9.720415,...,1.494893,2.008358,7.611064,2.355817,1.883806,437.438773,241.846078,4.083939,86.146223,6.756559
1001,2.704399,1.618410,40.131562,2.072599,448.924448,3.764906,5.237210,257.850076,2277.524139,9.752413,...,1.503261,2.027179,7.647048,2.367670,1.892955,442.181774,242.487306,4.101714,86.395904,6.824988


In [24]:
funds_date_2018

,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,Atmos Ações FIC FIA,AZ Quest Small Mid Caps FIC FIA,Bahia AM Smid Caps Valor FIC FIA,BNP Paribas Small Caps FIA,Bogari Value FIC FIA,Brasil Capital FIC FIA,...,Opportunity Selection FIC FIA,Pacifico Ações FIC FIA,Real Investor FIC FIA BDR Nível I,Sharp Equity Value Feeder FIC FIA,SPX Apache FIC FIA,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA
1003,2.777787,1.637691,40.785179,2.087327,452.111636,3.823985,5.315937,259.829941,2293.085280,9.858843,...,1.522436,2.044858,7.762725,2.386720,1.916621,443.160845,246.681836,4.128997,87.089007,6.872046
1004,2.775030,1.639682,40.856300,2.087340,451.550489,3.819891,5.321547,259.154815,2290.568933,9.842317,...,1.526785,2.050752,7.781479,2.383506,1.919479,443.781566,247.608300,4.137740,87.096967,6.874096
1005,2.817346,1.648092,41.305127,2.104275,453.724914,3.850029,5.380621,258.686600,2296.999764,9.851619,...,1.532176,2.050026,7.809388,2.383136,1.928057,445.309466,250.403191,4.150698,87.133637,6.876539
1006,2.808839,1.652186,41.308046,2.109934,455.473824,3.848304,5.377012,259.828587,2311.781883,9.907399,...,1.537560,2.060964,7.843674,2.394244,1.933261,447.928088,249.887855,4.161703,87.458319,6.906440
1007,2.809030,1.656281,41.343549,2.104291,455.922302,3.849644,5.391313,258.426214,2304.716969,9.879561,...,1.548134,2.059333,7.860284,2.393697,1.936502,449.581910,251.757801,4.150386,87.523889,6.923349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,3.486037,1.953573,50.221459,2.386577,520.183011,4.627033,5.990521,253.868804,2536.295221,10.686294,...,1.700511,2.302046,9.189205,2.730045,2.135467,555.790757,297.977340,4.408004,87.559855,7.973142
1248,3.486213,1.953454,50.215659,2.386498,520.163710,4.626761,5.989989,253.699585,2536.169971,10.685594,...,1.700426,2.301798,9.186111,2.722322,2.135387,555.724080,297.960173,4.401913,87.554579,7.972638
1249,3.420264,1.951558,50.079158,2.374598,519.202439,4.601407,5.942799,251.823471,2527.125502,10.691464,...,1.699242,2.300292,9.108459,2.733917,2.121019,555.674519,298.319433,4.394567,87.855351,7.966571
1250,3.424997,1.966922,50.452181,2.390661,527.198344,4.637101,5.981217,253.602616,2542.915225,10.744698,...,1.713592,2.313650,9.183511,2.757279,2.136203,563.719074,299.427962,4.426769,88.206946,8.002087


## Find weights for max Sharpe

In [25]:
def create_portfolios(df):
    funds_list = list(df['FUNDS'])
    length = len(funds_list)
    port_list = []
    
    for l in range(0,length):
        for k in range(l+1,length):
            for j in range(k+1,length):
                for i in range(j+1,length):
                    port_list.append([funds_list[l], funds_list[k], funds_list[j], funds_list[i]])

    return port_list

In [26]:
def weights_maxsharpe(port_list, mu, S, rf_rate):
    
    # creates a dataframe with covariances
    covar_port_df = pd.DataFrame(index = port_list, columns = port_list)
    
    for j in port_list:
        for i in port_list:
            covar_port_df.at[i, j] = S.at[i, j]

    # pd.Series() with the returns
    portSeries = pd.Series(index = port_list, dtype="float64")
    for fund in port_list:
        portSeries.at[fund] = mu.at[fund]

    muPort = portSeries.copy()
    SPort = covar_port_df.copy()
    
    # find weights for max Sharpe
    try:
        
        # convex solution for maximum Sharpe
        ef = EfficientFrontier(muPort, SPort)
        weights = ef.max_sharpe(risk_free_rate = rf_rate);
        cleaned_weights = ef.clean_weights()
    
    except:
        
        # non-convex solution for maximum Sharpe
        ef = EfficientFrontier(muPort, SPort)
        weights = ef.nonconvex_objective(
                  objective_functions.sharpe_ratio,
                  objective_args=(ef.expected_returns, ef.cov_matrix),
                  weights_sum_to_one=True);
        cleaned_weights = ef.clean_weights()

    return dict(cleaned_weights)

In [27]:
# build portfolios
all_portfolios = create_portfolios(funds_cnpj_df)

# covariances and returns dictionaries
mu = {k:mean_historical_return(val[1]) for k,val in funds_date_dict.items()}
S = {k:sample_cov(val[1]) for k,val in funds_date_dict.items()}


# find weights for max Sharpe in the training periods
weights_train_list = []
for window, period in windows_dict.items():
    train_years = period[:-1]
    for j in train_years:
        for port in all_portfolios:
            dicio = weights_maxsharpe(port, mu[j], S[j], rf_rate_dict[j][1])
            weights_train_list.append(list(dicio.keys()) + list(dicio.values()) + [j] + [window])


# find weights for max Sharpe in the prediction periods
weights_pred_list = []
for window, period in windows_dict.items():
    pred_year = [period[-1]]
    for j in pred_year:
        for port in all_portfolios:
            dicio = weights_maxsharpe(port, mu[j], S[j], rf_rate_dict[j][1])
            weights_pred_list.append(list(dicio.keys()) + list(dicio.values()) + [j] + [window])


# create training and prediction datasets
train_port_weights_df = pd.DataFrame(weights_train_list, columns=['F1', 'F2', 'F3', 'F4',
                                                                  'w_F1', 'w_F2', 'w_F3', 'w_F4',
                                                                  'period', 'window'])

pred_port_weights_df = pd.DataFrame(weights_pred_list, columns=['F1', 'F2', 'F3', 'F4',
                                                                'w_F1', 'w_F2', 'w_F3', 'w_F4',
                                                                'period', 'window'])


C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337

C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337

C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337

C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337

C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337

In [28]:
train_port_weights_df

,F1,F2,F3,F4,w_F1,w_F2,w_F3,w_F4,period,window
0,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,1.00000,0.0,0.0,0.0,20142015,0
1,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atmos Ações FIC FIA,0.99999,0.0,0.0,0.0,20142015,0
2,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,AZ Quest Small Mid Caps FIC FIA,1.00000,0.0,0.0,0.0,20142015,0
3,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Bahia AM Smid Caps Valor FIC FIA,1.00000,0.0,0.0,0.0,20142015,0
4,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,BNP Paribas Small Caps FIA,1.00000,0.0,0.0,0.0,20142015,0
...,...,...,...,...,...,...,...,...,...,...
431515,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,0.00000,0.0,0.0,1.0,20192020,3
431516,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,XP Investor FIA,0.00000,1.0,0.0,0.0,20192020,3
431517,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,VOKIN GBV,XP Investor FIA,0.00000,0.0,1.0,0.0,20192020,3
431518,Squadra Long Only FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA,0.00000,0.0,1.0,0.0,20192020,3


In [29]:
pred_port_weights_df

,F1,F2,F3,F4,w_F1,w_F2,w_F3,w_F4,period,window
0,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,0.0,0.0,0.11176,0.88824,20172018,0
1,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atmos Ações FIC FIA,0.0,0.0,0.00000,1.00000,20172018,0
2,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,AZ Quest Small Mid Caps FIC FIA,0.0,0.0,0.00000,1.00000,20172018,0
3,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Bahia AM Smid Caps Valor FIC FIA,0.0,0.0,0.00000,1.00000,20172018,0
4,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,BNP Paribas Small Caps FIA,0.0,0.0,0.00000,1.00000,20172018,0
...,...,...,...,...,...,...,...,...,...,...
143835,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,0.0,0.0,0.00000,1.00000,20202021,3
143836,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,XP Investor FIA,0.0,0.0,0.00000,1.00000,20202021,3
143837,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,VOKIN GBV,XP Investor FIA,0.0,0.0,0.00000,1.00000,20202021,3
143838,Squadra Long Only FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA,0.0,0.0,0.00000,1.00000,20202021,3


## Builds the training and prediction datasets

In [30]:
def dataset_header():
    head_funds = ['F1','F2','F3','F4']
    comb_funds = [['F1','F2'],['F1','F3'],['F1','F4'],['F2','F3'],['F2','F4'],['F3','F4']]
    head_returns = ['ret_F1','ret_F2','ret_F3','ret_F4']
    head_risks = ['rsk_F1','rsk_F2','rsk_F3','rsk_F4']
    head_corr = ['CORR_F1F2','CORR_F1F3','CORR_F1F4','CORR_F2F3','CORR_F2F4','CORR_F3F4']
    head_weights = ['w_F1','w_F2','w_F3','w_F4','period','window']
    
    header = head_funds + head_returns + head_risks + head_corr + head_weights

    return header, head_funds, comb_funds, head_returns, head_risks, head_corr

In [31]:
# datasets columns
header, head_funds, comb_funds, head_returns, head_risks, head_corr = dataset_header()

# inputs target weights in the datasets
train_dataset = pd.DataFrame(columns=header)
pred_dataset = train_dataset.copy()

for column in train_port_weights_df.columns:
    train_dataset[column] = train_port_weights_df[column]
    pred_dataset[column] = pred_port_weights_df[column]


    
# fills training dataset with the risks, returns, and correlations
for idx in train_dataset.index:
    
    for fund, ret_col, rsk_col in zip(head_funds, head_returns, head_risks):        
        train_dataset.at[idx,ret_col] = funds_return_dict[train_dataset.at[idx,"period"]][0][train_dataset.at[idx,fund]]
        train_dataset.at[idx,rsk_col] = funds_risk_dict[train_dataset.at[idx,"period"]][0][train_dataset.at[idx,fund]]
      
    for pair, corr_col in zip(comb_funds, head_corr):
        train_dataset.at[idx, corr_col] = corr_dict[train_dataset.at[idx,"period"]][0].at[train_dataset.at[idx,pair[0]],train_dataset.at[idx,pair[1]]]


        
# fills prediction dataset with the risks, returns, and correlations
for idx in pred_dataset.index:
    
    for fund, ret_col, rsk_col in zip(head_funds, head_returns, head_risks):
        pred_dataset.at[idx,ret_col] = funds_return_dict[pred_dataset.at[idx,"period"]][0][pred_dataset.at[idx,fund]]
        pred_dataset.at[idx,rsk_col] = funds_risk_dict[pred_dataset.at[idx,"period"]][0][pred_dataset.at[idx,fund]]
        
    for pair, corr_col in zip(comb_funds, head_corr):
        pred_dataset.at[idx, corr_col] = corr_dict[pred_dataset.at[idx,"period"]][0].at[pred_dataset.at[idx,pair[0]],pred_dataset.at[idx,pair[1]]]


In [32]:
train_dataset.iloc[-10:,:]

,F1,F2,F3,F4,ret_F1,ret_F2,ret_F3,ret_F4,rsk_F1,rsk_F2,...,CORR_F1F4,CORR_F2F3,CORR_F2F4,CORR_F3F4,w_F1,w_F2,w_F3,w_F4,period,window
431510,SPX Apache FIC FIA,Squadra Long Only FIC FIA,VOKIN GBV,XP Investor FIA,0.120099,0.239816,0.343334,0.200792,0.310704,0.340492,...,0.955317,0.898935,0.957894,0.902033,0.0,0.0,1.0,0.0,20192020,3
431511,SPX Apache FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,0.120099,0.139883,0.322755,0.343334,0.310704,0.326045,...,0.915881,0.897886,0.886444,0.88881,0.0,0.0,0.0,1.0,20192020,3
431512,SPX Apache FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,XP Investor FIA,0.120099,0.139883,0.322755,0.200792,0.310704,0.326045,...,0.955317,0.897886,0.940402,0.939227,0.0,1.0,0.0,0.0,20192020,3
431513,SPX Apache FIC FIA,Tempo Capital Manacá FIC FIA,VOKIN GBV,XP Investor FIA,0.120099,0.139883,0.343334,0.200792,0.310704,0.326045,...,0.955317,0.886444,0.940402,0.902033,0.0,0.0,1.0,0.0,20192020,3
431514,SPX Apache FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA,0.120099,0.322755,0.343334,0.200792,0.310704,0.308922,...,0.955317,0.88881,0.939227,0.902033,0.0,0.0,1.0,0.0,20192020,3
431515,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,0.239816,0.139883,0.322755,0.343334,0.340492,0.326045,...,0.898935,0.897886,0.886444,0.88881,0.0,0.0,0.0,1.0,20192020,3
431516,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,XP Investor FIA,0.239816,0.139883,0.322755,0.200792,0.340492,0.326045,...,0.957894,0.897886,0.940402,0.939227,0.0,1.0,0.0,0.0,20192020,3
431517,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,VOKIN GBV,XP Investor FIA,0.239816,0.139883,0.343334,0.200792,0.340492,0.326045,...,0.957894,0.886444,0.940402,0.902033,0.0,0.0,1.0,0.0,20192020,3
431518,Squadra Long Only FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA,0.239816,0.322755,0.343334,0.200792,0.340492,0.308922,...,0.957894,0.88881,0.939227,0.902033,0.0,0.0,1.0,0.0,20192020,3
431519,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA,0.139883,0.322755,0.343334,0.200792,0.326045,0.308922,...,0.940402,0.88881,0.939227,0.902033,0.0,0.0,1.0,0.0,20192020,3


In [33]:
pred_dataset

,F1,F2,F3,F4,ret_F1,ret_F2,ret_F3,ret_F4,rsk_F1,rsk_F2,...,CORR_F1F4,CORR_F2F3,CORR_F2F4,CORR_F3F4,w_F1,w_F2,w_F3,w_F4,period,window
0,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atlas One FIC FIA,0.475044,0.269558,0.274398,0.222275,0.384417,0.198061,...,0.753389,0.955096,0.937329,0.928328,0.0,0.0,0.11176,0.88824,20172018,0
1,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Atmos Ações FIC FIA,0.475044,0.269558,0.274398,0.207363,0.384417,0.198061,...,0.722829,0.955096,0.869817,0.877949,0.0,0.0,0.00000,1.00000,20172018,0
2,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,AZ Quest Small Mid Caps FIC FIA,0.475044,0.269558,0.274398,0.342387,0.384417,0.198061,...,0.779834,0.955096,0.932448,0.897373,0.0,0.0,0.00000,1.00000,20172018,0
3,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,Bahia AM Smid Caps Valor FIC FIA,0.475044,0.269558,0.274398,0.271194,0.384417,0.198061,...,0.773381,0.955096,0.958631,0.944569,0.0,0.0,0.00000,1.00000,20172018,0
4,Alaska Black FIC FIA BDR Nível I,Apex Ações 30 FIC FIA,ARX Income FIC FIA,BNP Paribas Small Caps FIA,0.475044,0.269558,0.274398,0.211564,0.384417,0.198061,...,0.736238,0.955096,0.857497,0.851407,0.0,0.0,0.00000,1.00000,20172018,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143835,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,VOKIN GBV,-0.052536,0.012645,-0.067776,0.108509,0.35261,0.335134,...,0.904303,0.872107,0.895256,0.885927,0.0,0.0,0.00000,1.00000,20202021,3
143836,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,Velt FIC FIA,XP Investor FIA,-0.052536,0.012645,-0.067776,-0.095097,0.35261,0.335134,...,0.948525,0.872107,0.928567,0.930813,0.0,0.0,0.00000,1.00000,20202021,3
143837,Squadra Long Only FIC FIA,Tempo Capital Manacá FIC FIA,VOKIN GBV,XP Investor FIA,-0.052536,0.012645,0.108509,-0.095097,0.35261,0.335134,...,0.948525,0.895256,0.928567,0.896683,0.0,0.0,0.00000,1.00000,20202021,3
143838,Squadra Long Only FIC FIA,Velt FIC FIA,VOKIN GBV,XP Investor FIA,-0.052536,-0.067776,0.108509,-0.095097,0.35261,0.335904,...,0.948525,0.885927,0.930813,0.896683,0.0,0.0,0.00000,1.00000,20202021,3


## Download datasets

In [34]:
train_dataset.to_csv(r'./trainpred_dataset/FIA1_20142022_train_dataset.csv',
                     encoding = 'utf-8-sig',
                     index = False)

pred_dataset.to_csv(r'./trainpred_dataset/FIA1_20142022_pred_dataset.csv',
                    encoding = 'utf-8-sig',
                    index = False)